## Analysis of flight delays and taxiing times and its relationship to the local weather for Jan 2018

In this notebook, I am experimenting with combining data from different sources to find correlations between relevant parameters and experimenting with creating interactive plots using Bokeh.  The data and the correponding data sources used in this analysis are

* Flight details are obtained from [On-Time Flight performance database](https://www.transtats.bts.gov/DL_SelectFields.asp?DB_Short_Name=On-Time&Table_ID=236)
* Airline details are obtained from [Carrier Decode Table](https://www.transtats.bts.gov/Tables.asp?DB_ID=595&DB_Name=Aviation%20Support%20Tables&DB_Short_Name=Aviation%20Support%20Tables#)
* Airport information (including precise location) is obtained from [Master Coordinate Table](https://www.transtats.bts.gov/Tables.asp?DB_ID=595&DB_Name=Aviation%20Support%20Tables&DB_Short_Name=Aviation%20Support%20Tables#)
* Daily average weather conditions are obtained from [National Climatic Data Center (NCDC)](ftp://ftp.ncdc.noaa.gov/pub/data/gsod)
* Information (including precise location) about weather stations are obtained from [Station list](ftp://ftp.ncdc.noaa.gov/pub/data/noaa/isd-history.txt)

All data presented here only correspond to airports within the US for the month of January 2018.  Data for other time periods (month, year) are availabe at the sites mentioned above.  Similar analysis could be applied to those other time periods by downloading the appropriate data sets.  But, that is beyond the scope of this analysis.

#### (For now, I am not including any weather related plots.  These will be added later).

In [17]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [1]:
#######################################################################
#  Import required libraries and Delete all pre-defined variables
#######################################################################
import numpy as Numpy
#import bokeh as Bokeh
import bokeh.plotting as BokehPlotting
import bokeh.io as BokehIO
import bokeh.palettes as BokehPalettes
import bokeh.models as BokehModels
import bokeh.layouts as BokehLayouts
import bokeh.models.widgets as BokehWidgets
import bokeh.core.properties as BokehCoreProps
#import scipy as Scipy
#import pylab as Pylab
import matplotlib.pyplot as Plt
import pandas as Pandas
import math as Math
#import time as Time
import ftplib as Ftplib
import pickle as Pickle
import os as Os
import datetime as DateTime
for name in dir():
    if not name.startswith('_'):
        del name
Pandas.set_option('display.expand_frame_repr', False)

In [2]:
#######################################################################
# User Input
#######################################################################
baseDir = 'datasets/'
yyyymmOfInt = '201801'
airportInfoFile = 'Airport_locations.csv'
airlinesFile = 'Carriers.csv'
weatherStatLocFile = 'isd-history.txt'
weatherServer = 'ftp.ncdc.noaa.gov' 
weatherPath = 'pub/data/gsod/'
weatherUser = 'anonymous'
weatherServer = 'local' # if weather files are going to be local
flightFile = 'Flights_onTime_' + yyyymmOfInt + '.pickle'
yearOfInt = yyyymmOfInt[0:4]
TOOLS = "hover,pan,zoom_in,zoom_out,box_zoom,reset,save"
flightColsFull = ['Date', 'CAR', 'TAIL', 'FLNum', 'ORIGIN_ID', 'ORIGIN', 'DEST_ID', 'DEST', 
              'DEP_SCH', 'DEP_ACT', 'DEP_DEL', 'DEP_TAXI', 'DEP_OFF',
              'ARR_ON', 'ARR_TAXI', 'ARR_SCH', 'ARR_ACT', 'ARR_DEL',
              'CANCEL', 'CANCEL_CODE', 'DIV', 'FLY_SCH', 'FLY_ACT', 'FLY_AIR', 'FLY_DIST',
              'DEL_CAR', 'DEL_WET', 'DEL_NAS', 'DEL_SEC', 'DEL_AIR']
flightColNos = [0, 1, 4, 6, 10, 11, 14, 17]
flightCols = ['Date', 'CAR', 'AIRPORT_ID', 'DURATION']
airportInfoColNos = [0,3,4,18,23]
airportInfoCols = ['AIRPORT_ID', 'Airport', 'City', 'LAT', 'LON']

In [3]:
#######################################################################
# User defined functions
#######################################################################
def distance(loc1, loc2):
    # Uses the Haversine formula to compute and return the distance (in km) between 
    #   'loc1' = [lat1, lon1] and 'loc2' = [lat2, lon2]
    p = (Math.pi/180)  #Pi/180
    a = 0.5 - Math.cos((loc2[0]-loc1[0])*p)/2 + Math.cos(loc1[0]*p)*Math.cos(loc2[0]*p) * (1-Math.cos((loc2[1]-loc1[1])*p)) / 2
    return 12742 * Math.asin(Math.sqrt(a)) # 2 * R; R = 6371 km

def is_float(x):
    try:
        float(x)
        return True
    except:
        return False
    
def nearest_Station(loc):
    # given the data frame 'weatherStations' that contains 'STN' 'WBAN', and 'LOC' = [LAT, LON]
    # find the weather station that is closest to 'loc' = [LAT, LON]
    # and return the a list of 'STN', 'WBAN', and 'distance to the closest station'
    
    #weatherStations = weatherStatData
    #loc = (39.86166667, -104.67305556)
    
    localWeatherStat = weatherStatData.copy()
    localWeatherStat['dist'] = localWeatherStat['LOC'].apply(lambda x: distance([float(x[0]), float(x[1])], [loc[0], loc[1]]))
    localWeatherStat = localWeatherStat.sort_values(['dist'], ascending=['False']).iloc[0,]
    
    return [localWeatherStat['STN'], 
            format(localWeatherStat['WBAN'],'05d'), 
            round(localWeatherStat['dist'],0)]

In [4]:
#######################################################################
# Read Flight data
#######################################################################
Os.system('gunzip ' + Os.path.join(baseDir + flightFile) + '.gz')
flightData = Pandas.read_pickle(Os.path.join(baseDir + flightFile))

In [5]:
#######################################################################
# Combine arrival and departure delays and arrival and departure taxi times
#######################################################################
flightDepDel = flightData[['Date', 'CAR', 'ORIGIN_ID', 'DEP_DEL']].copy()
flightArrDel = flightData[['Date', 'CAR', 'DEST_ID', 'ARR_DEL']].copy()
flightDepTaxi = flightData[['Date', 'CAR', 'ORIGIN_ID', 'DEP_TAXI']].copy()
flightArrTaxi = flightData[['Date', 'CAR', 'DEST_ID', 'ARR_TAXI']].copy()
flightDepDel.columns = flightArrDel.columns = flightDepTaxi.columns = flightArrTaxi.columns = flightCols
flightDepDel['Type'] = 'DEP_DEL'
flightArrDel['Type'] = 'ARR_DEL'
flightDepTaxi['Type'] = 'DEP_TAXI'
flightArrTaxi['Type'] = 'ARR_TAXI'
flightDur = Pandas.concat([flightDepDel, flightArrDel, flightDepTaxi, flightArrTaxi])
del flightData, flightDepDel, flightArrDel, flightDepTaxi, flightArrTaxi

In [6]:
#######################################################################
# Only select those airlines that have a large number of flights (at least 2 per hour on average)
# Merge this info with airline names
#######################################################################
topAirlines = flightDur.groupby(['CAR', 'Type'])['DURATION'].agg(['count']).reset_index().rename(columns={'count':'AirlMonthTotal'})
topAirlines = topAirlines.loc[topAirlines['AirlMonthTotal'] > 24*2*flightDur['Date'].nunique()]
#
airlineData = Pandas.read_csv(Os.path.join(baseDir + airlinesFile), index_col=False)
airlineData.columns = ['CAR', 'Airline']
topAirlines = topAirlines.merge(airlineData, on=['CAR'], how='inner')
del airlineData

In [7]:
#######################################################################
# Only select those airports that have a large number of flights (at least 2 per hour on average)
# Merge this with airport info 
#######################################################################
topAirports = flightDur.groupby(['AIRPORT_ID', 'Type'])['DURATION'].agg(['count']).reset_index().rename(columns={'count':'AirpMonthTotal'})
topAirports = topAirports.loc[topAirports['AirpMonthTotal'] > 24*2*flightDur['Date'].nunique()]
#
airportData = Pandas.read_csv(Os.path.join(baseDir + airportInfoFile), index_col=False)
airportData = airportData.iloc[:,airportInfoColNos].copy()
airportData.columns = airportInfoCols
airportData = airportData.dropna(subset=['LAT', 'LON'])
topAirports = topAirports.merge(airportData, on=['AIRPORT_ID'], how='inner')
del airportData

In [8]:
#######################################################################
# Extract the location of the weather stations that are active during the time period
# of the flight delays.  Make sure to include only those data that have valid LAT and LON
#######################################################################
weatherStatData = Pandas.read_fwf(Os.path.join(baseDir + weatherStatLocFile), 
                                  names = ['STN', 'WBAN', 'STN_Name', 'CTRY', 'ST', 'CALL', 
                                           'LAT', 'LON', 'ELEV_M', 'BEGIN', 'END'],
                                           header = None,
                                           colspecs = [(0,6), (7,12), (13,42), (43,47), (48,50), (51,56), 
                                                       (57,64), (65,73), (74,81), (82,90), (91,99)], 
                                           delim_whitespace=True, skiprows=22)
weatherStatData = weatherStatData.loc[(weatherStatData['BEGIN'] < int(yyyymmOfInt + '01')) & 
                                      (weatherStatData['END'] > int(yyyymmOfInt + '31'))]
weatherStatData = weatherStatData.dropna(subset=['LAT', 'LON'])
weatherStatData = weatherStatData.loc[(weatherStatData['LAT'].apply(lambda x: is_float(x))) &
                                      (weatherStatData['LON'].apply(lambda x: is_float(x)))]
weatherStatData['LOC'] = list(zip(weatherStatData['LAT'], weatherStatData['LON']))
weatherStatData.drop(['CTRY', 'ST', 'CALL', 'BEGIN', 'END', 'LAT', 'LON'], axis=1, inplace=True)

In [9]:
#######################################################################
# For each of the airports in 'topAirports', find the nearest weather station 
#######################################################################
airpWet = topAirports.loc[topAirports['Type']=='DEP_DEL', ['AIRPORT_ID', 'LAT', 'LON']]
airpWet['LOC'] = list(zip(airpWet['LAT'], airpWet['LON']))
airpWet['LOC'] = airpWet['LOC'].apply(lambda x: nearest_Station(x))
airpWet['statName'] = airpWet['LOC'].apply(lambda x: x[0] + '-' + x[1] + '-' + yearOfInt + '.op.gz')
airpWet['dist2WetStat'] = airpWet['LOC'].apply(lambda x: x[2])
airpWet.drop(['LAT', 'LON', 'LOC'], axis=1, inplace=True)    
topAirports = topAirports.merge(airpWet, on=['AIRPORT_ID'], how='inner')
del weatherStatData, airpWet

In [10]:
#######################################################################
# Merge flightDur with topAirports and topAirlines
#######################################################################
flightDur = flightDur.merge(topAirports, on=['AIRPORT_ID', 'Type'], how='inner')
flightDur = flightDur.merge(topAirlines, on=['CAR', 'Type'], how='inner')
del topAirports, topAirlines
airportList = list(flightDur.sort_values(['City'], ascending=['True'])['City'].unique())
airlineList = list(flightDur.sort_values(['Airline'], ascending=['True'])['Airline'].unique())

In [12]:
#######################################################################
# These will be the base data for rest of the analysis
# Select a default airport and airline 
#######################################################################
airport = [x for x in airportList if 'Denver' in x][0]
airline = [x for x in airlineList if 'United' in x][0]
statName = flightDur.loc[flightDur['City']==airport,'statName'].unique()[0]

In [13]:
#######################################################################
# Styling for a plot
#######################################################################
def style(p):
    # Title 
    p.title.align = 'center'
    p.title.text_font_size = '14pt'
    p.title.text_font = 'Helvetica'

    # Axis titles
    p.xaxis.axis_label_text_font_size = '18pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '18pt'
    p.yaxis.axis_label_text_font_style = 'bold'

    # Tick labels
    p.xaxis.major_label_text_font_size = '14pt'
    p.yaxis.major_label_text_font_size = '14pt'

    return p

In [16]:
airport = 'Denver, CO'

airpByDatestats = flightDur.groupby(['Date', 'Type', 'City', 'AirpMonthTotal'])['DURATION'].agg(['mean', 'median', 'count']).reset_index()
airpByDatestats['Day'] = airpByDatestats['Date'].apply(lambda x: int(x[8:10]))
airpByDatestats.drop(['Date'], axis=1, inplace=True)

DepDel = airpByDatestats.loc[airpByDatestats['Type']=='DEP_DEL'].copy()
ArrDel = airpByDatestats.loc[airpByDatestats['Type']=='ARR_DEL'].copy()
DepDel.drop(['Type'], axis=1, inplace=True)
ArrDel.drop(['Type'], axis=1, inplace=True)

CDSDepDel_All = BokehModels.ColumnDataSource(DepDel)
CDSArrDel_All = BokehModels.ColumnDataSource(ArrDel)

CDSDepDel = BokehModels.ColumnDataSource(DepDel.loc[DepDel['City']==airport].copy())
CDSArrDel = BokehModels.ColumnDataSource(ArrDel.loc[ArrDel['City']==airport].copy())

BokehIO.output_notebook()

#######################################################################
plt = BokehPlotting.figure(
                             title="Delays by Day", 
                             x_axis_label='Day of month', 
                             y_axis_label='Delay (mins)',
                             plot_width=900, plot_height=600,
                             tools = TOOLS,
                             tooltips = [('Day', '@Day'),
                                         ('Avg', '@mean'),
                                         ('Med', '@median'),
                                         ('Count', '@count')]    
                             )
plt.circle(x = 'Day', y = 'mean', line_color='red' ,color = 'red', fill_alpha=1, size=7,
                  source = CDSDepDel, legend='Dep')

plt.circle(x = 'Day', y = 'mean', line_color='blue' ,color = 'blue', fill_alpha=1, size=7,
              source = CDSArrDel, legend='Arr')

plt.legend.location = 'top_left'
plt.legend.orientation = "horizontal"

plt = style(plt)
#######################################################################

airpInp = BokehWidgets.Select(title="Select Airport city", value=airport, options=airportList)

update_plot = BokehModels.CustomJS(args=dict(DSDepDel=CDSDepDel_All, PSDepDel=CDSDepDel, 
                             DSArrDel=CDSArrDel_All, PSArrDel=CDSArrDel, 
                             widInp=airpInp), code="""
    var DepDel_All = DSDepDel.data
    var ArrDel_All = DSArrDel.data    
    var DepDel = PSDepDel.data
    var ArrDel = PSArrDel.data    
    var airport = widInp.value
    
    Day_All = DepDel_All['Day']
    City_All = DepDel_All['City']
    AirpMonthTotal_All = DepDel_All['AirpMonthTotal']
    mean_All = DepDel_All['mean']
    median_All = DepDel_All['median']
    count_All = DepDel_All['count']
    
    Day = DepDel['Day']
    City = DepDel['City']
    AirpMonthTotal = DepDel['AirpMonthTotal']
    mean = DepDel['mean']
    median = DepDel['median']
    count = DepDel['count']

    j = 0
    for (i = 0; i < City_All.length; i++) {
        if (City_All[i] == airport) {
            Day[j] = Day_All[i]
            City[j] = City_All[i]
            AirpMonthTotal[j] = AirpMonthTotal_All[i]
            mean[j] = mean_All[i]
            median[j] = median_All[i]
            count[j] = count_All[i]
            j++
        }
    }
    
    Day_All = ArrDel_All['Day']
    City_All = ArrDel_All['City']
    AirpMonthTotal_All = ArrDel_All['AirpMonthTotal']
    mean_All = ArrDel_All['mean']
    median_All = ArrDel_All['median']
    count_All = ArrDel_All['count']
    
    Day = ArrDel['Day']
    City = ArrDel['City']
    AirpMonthTotal = ArrDel['AirpMonthTotal']
    mean = ArrDel['mean']
    median = ArrDel['median']
    count = ArrDel['count']

    j = 0
    for (i = 0; i < City_All.length; i++) {
        if (City_All[i] == airport) {
            Day[j] = Day_All[i]
            City[j] = City_All[i]
            AirpMonthTotal[j] = AirpMonthTotal_All[i]
            mean[j] = mean_All[i]
            median[j] = median_All[i]
            count[j] = count_All[i]
            j++
        }
    }

    PSDepDel.change.emit()
    PSArrDel.change.emit()
""")

airpInp.js_on_change('value', update_plot)

BokehPlotting.show(BokehLayouts.column(airpInp, plt))

Loading BokehJS ...